## 卷积神经网络 Task7

* 卷积运算的定义、动机（稀疏权重、参数共享、等变表示）。一维卷积运算和二维卷积运算。
* 反卷积(tf.nn.conv2d_transpose)
* 池化运算的定义、种类（最大池化、平均池化等）、动机。
* Text-CNN的原理。
* 利用Text-CNN模型来进行文本分类。



In [1]:
'''
利用Text-CNN模型来进行文本分类。
'''
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import re
import time
import datetime
from tensorflow.contrib import learn
#初始化tensorflow 读取数据类

In [44]:
#==【读数据模块】==
def load_data_and_labels(posiData,negaData):
    
    positiveText = open(posiData,'rb').read().decode('utf-8')
    negativeText = open(negaData,'rb').read().decode('utf-8')
    
    #拆分文本集
    positive_cases = positiveText.split('\n')[:-1]
    negative_cases = negativeText.split('\n')[:-1]
    #邮件集以\n分隔数组，清除最后一个空元素
    
    positive_cases = [s.strip() for s in positive_cases]#清除文档两端空格
    negative_cases = [s.strip() for s in negative_cases]
   

    #组合特征集X_text与labels分类y
    X_text = positive_cases + negative_cases#正负例混合
    X_text = [clean_str(sent) for sent in X_text]#清除空格
    
    #labels定义，在正例中设为onehot矩阵[0,1]负例中为[1,0]，两类两列
    positive_label = [[0,1] for _ in positive_cases]
    negative_label = [[1,0] for _ in negative_cases]
    #下划线表示 临时变量，仅用一次,只要循环，不取值
    y = np.concatenate([positive_label,negative_label],0)#垂直方向合并
    
    return [X_text,y]
    
    
#清洗数据，标点符号之类的过滤  
def clean_str(string):

    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()
    
#定义训练迭代函数
def batch_iter(data,batch_size,num_epoch,shuffle=True):
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) +1
    #epoche 所有数据都迭代一次，
    #每个里面有batch
    #一个epoch里有多少个batchsize,+1是为了保本一些
    
    #遍历epoch组
    for epoch in range(num_epoch):
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            #permutation，洗牌并返回一个洗牌后的矩阵副本索引
            shuffle_data = data[shuffle_indices]#//方法：在函数中先用data指代完成操作，再传参进来
        else:
            shuffle_data = data
        #遍历每个epoch里的batch组
        for batch_num in range(num_batches_per_epoch):
            #定义数据切片起始点
            start_index = batch_num*batch_size
            end_index = min((batch_num+1)*batch_size,data_size)#比较看是不是文章末尾了
            
            yield shuffle_data[start_index:end_index]
            
    

In [29]:
#==【构造CNN网络结构】==

class TextCNN():
    def __init__(self,sequence_length,num_classes,vocab_size,
                 embedding_size,filter_size,num_filters,l2_reg_lambda=0.0):
        #初始化
        self.input_x = tf.placeholder(tf.int32,[None,sequence_length],name = 'input_x')
        #输入特征x，长度为句长
        self.input_y = tf.placeholder(tf.int32,[None,num_classes],name='input_y')
        #输入labels y，长度为分类长度
        self.dropout_keep_prob = tf.placeholder(tf.float32,name = 'dropout')
        
        #正则惩罚项，可先写为常量
        l2_loss = tf.constant(0.0)#0.0小数点，定义常数，一个数，不是0,0
        
        #==embedding层(向量化嵌入矩阵，隐层)处理==
        with tf.device('/cpu:0'),tf.name_scope('embedding'):
            #初始化w权重
            self.W = tf.Variable(tf.random_uniform([vocab_size,embedding_size],-1.0,1.0,name='W'))
            #vocab_size文本最大长度（文档不重复词向量onehot编码1万维），映射成embedding_size 128维
            #-1.0 1.0向量编码初始化范围 uniform不重复的
            
            #embedding_lookup窗口滑动取单词，把输入词转成向量（嵌到上面初始化过的W矩阵）
            self.embedded_chars = tf.nn.embedding_lookup(self.W,self.input_x)#tf.nn.embedding_lookup 别少了up
            #上面组合了滑动窗口宽高，再加一维滑动窗口深度filter变四维
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars,-1)
            
            #以上完成了从输入文档的1万维词转为128维向量的操作
            
        #==卷积层+最大池化层处理==
        pooled_outputs = []
        
        #循环遍历不同大小的filter_size，即一次滑动组装多少个单词， 参数数定义了3，4，5三种
        for i,f_size in enumerate(filter_size):
            #在命名域中构造不同个数的单词组合向量
            with tf.name_scope('conv-maxpool-%s'%f_size):
                #写filter shape
                filter_shape = [f_size,embedding_size,1,num_filters]
                #参数[filter_height, filter_width, in_channels, out_channels]
                #[filter高一次滑动取多少个单词，filter宽词向量维度128，深度chanel通道如图象的RGB深度是3个 NLP没有深度默认为1，输出特征图数]
                W = tf.Variable(tf.truncated_normal(filter_shape,stddev = 0.1),name='W')
                #tf.truncated_normal高斯初始化（初始化为符合高斯正太分布的；stddev = 0.1标准，学习率
                #初始化偏移量b为常数即可,shape 1维的 有多少输出特征图就有多少个b
                b = tf.Variable(tf.constant(0.1,shape=[num_filters]),name='b')
                
                #代入以上参数，tf.nn.conv2d开始卷积
                conv = tf.nn.conv2d(self.embedded_chars_expanded,
                                    W,
                                    strides = [1,1,1,1],
                                    padding = 'VALID',
                                    name = 'conv'
                                    )
                #第一参：要卷积的对象（组装好的宽高深 四维输入词向量隐层）
                #strides滑动步长
                #padding不足filter窗口大小时是否填充补列，上面已补为最长单词大小，不需要补了，
                #SAME补充，VALID舍弃
                
                #每个卷积层都跟一个relu激活函数,
                h = tf.nn.relu(tf.nn.bias_add(conv,b),name='relu')
                #relu里面执行了一个wx+b，relu函数把线性回归转成了非线性非连续性分类概率
                
                #最大池化层（压缩特征图，以取最大值代表的方式）
                pooled = tf.nn.max_pool(h,
                    ksize = [1,sequence_length - f_size + 1,1,1],
                    strides = [1,1,1,1],
                    padding = 'VALID',
                    name = 'pool'
                )
                #h，连前面卷积的输出，
                #ksize 池化窗口大小，四维向量，一般是[1, height, width, 1]，
                #第1和第4参是batch和channels，因为不在batch和channels上做池化，所以这两个维度设为了1
                #最终输出的特征图高度算法公式h句长-filter+2padding/stride 即 sequence_length-f_size+1
                
                #池化结果存入池化层列表拼合成完整特征图
                pooled_outputs.append(pooled)
                
        #把特征图矩阵拉平，让FC全连接层可以接上
        #self.h_pool = tf.concat(3,pooled_outputs)#tf 1.8报错
        self.h_pool = tf.concat(pooled_outputs,3)
        #concat(3 在第4个维度连接，即深度，多个池化层整体拼成一个。
        #拼batch和高宽都是不对的，那些已处理完了，整体把分着的池化层拼起来，即第4维深度。0为第1维
        #定义总池化层个数，即总的filter个数一样的，池化是从filter挨个浓缩来的。
        num_filters_total = num_filters *len(filter_size)
        self.h_pool_flat = tf.reshape(self.h_pool,[-1,num_filters_total])

        #给上面结果加一层dropout
        with tf.name_scope('dropout'):
            self.h_drop = tf.nn.dropout(self.h_pool_flat,self.dropout_keep_prob)
            #dropout_keep_prob dropout的比例
        
        #全连接输出层初始化w
        with tf.name_scope('output'):
            W = tf.get_variable('W',
            shape = [num_filters_total,num_classes],
            initializer = tf.contrib.layers.xavier_initializer()
            )
            #shape 输入：拉平的pool层，输出：分类数
            #initializer初始化
            #定义b(之前竟然忘记了)，注意这个b的shape和分类数一样，输出层对应的是输出分类个b
            b = tf.Variable(tf.constant(0.1,shape=[num_classes]),name='b')
           
            l2_loss += tf.nn.l2_loss(W)#W的正则惩罚
            l2_loss += tf.nn.l2_loss(b)#b的正则惩罚

            #得分
            self.scores = tf.nn.xw_plus_b(self.h_drop,W,b,name='scores')
            # tf.nn.xw_plus_b((x, weights) + biases)函数
            #wx plus加 b
            #相当于matmul(x, weights) + biases
            #h_drop drop后的最终特征图集

            #预测
            self.predictions = tf.argmax(self.scores,1,name = 'predictions')
            #1，axis，argmax返回每一行中的最大位置索引，
            #即self.predictions是一个scores得分最高的对应的索引，
            #后面用于和得分比较，一致的即预测对了

        #损失函数层（softmax分类层）
        with tf.name_scope('loss'):
            #softmax_cross_entropy_with_logits不要少了_with_logits
            losses = tf.nn.softmax_cross_entropy_with_logits(
                logits=self.scores,labels = self.input_y)
            #softmax交叉熵 （预测分，labels或target输入的分类结果）
            self.loss = tf.reduce_mean(losses)+l2_loss*l2_reg_lambda
            #求loss平均
         
        #精确度
        with tf.name_scope('acuracy'):
            correct_predictions = tf.equal(self.predictions,tf.argmax(self.input_y,1))
            #tf.argmax(self.input_y,1)两个参数，不要写成tf.argmax(self.input_y),1
            #预测值得分最高值索引与输入值最大相同，则表示预测正确。axis=1按行
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions,'float'),name='accuracy')
            #算平均准确率，
            #cast bool型False True转数值型 0 1，转成1 0 才能计算平均值
                        
#TextCNN类完
'''
【==步骤总结：==
预处理数据
转成128维向量
多个n-gram filter
遍历filter
卷积、池化pool
全连接
最后得到结果】
'''

'\n【==步骤总结：==\n预处理数据\n转成128维向量\n多个n-gram filter\n遍历filter\n卷积、池化pool\n全连接\n最后得到结果】\n'

In [4]:
#==【train模块 - 参数字义】==

#==设计参数 用tensorflow标准格式，可以直接使用像类中的self.属性，（不用直接函数中写传的方式
#tf.flags.DEFINE_...定义参数（名字，默认值，描述）

#==数据参数==
tf.flags.DEFINE_float('dev_sample_percentage',.1,'训练集的比例')#.1 10%,别忘了点
tf.flags.DEFINE_string('positive_data_file','mail/rt-polarity.pos','正例数据路径')
tf.flags.DEFINE_string('negative_data_file','mail/rt-polarity.neg','负例数据路径')

#==神经网络参数==
tf.flags.DEFINE_integer('embedding',128,'隐层维度')
#输入单词的向量维度128列
tf.flags.DEFINE_string('filter_size','3,4,5','region滑动窗口大小')
#每次滑动组装3，4或5个单词组成多组filter，相当于组合n-gram
tf.flags.DEFINE_integer('num_filters',128,'filter数量')
#卷积后想要多少特征图
tf.flags.DEFINE_float('dropout',0.5,'dropout无用神经元弃用比')
tf.flags.DEFINE_float('L2_reg_lambda',0.0,'L2正则惩罚项')

#==训练参数==
tf.flags.DEFINE_integer('batch_size',32,'模型训练迭代分组数')
tf.flags.DEFINE_integer('num_epochs',200,'模型训练迭代轮数')
#迭代200轮，每轮把所有64组遍历一次
tf.flags.DEFINE_integer('evaluate_every',100,'打印间隔数')
#迭代100次打印一次
tf.flags.DEFINE_integer('checkpoint_every',100,'模型保存间隔')
#迭代100次保存一次模型
tf.flags.DEFINE_integer('num_checkpoints',5,'模型最多保存多少条？')

#==模型训练设备相关配置==
tf.flags.DEFINE_boolean('allow_soft_placement',True,'tf自由选择设备')
tf.flags.DEFINE_boolean('log_device_placement',True,'获取tf自动指派的设备并打印')

tf.app.flags.DEFINE_string('f', '', 'kernel')
#Unknown command line flag 'f' BUG解决

#参数解析
FLAGS = tf.flags.FLAGS
#以下把参数变字典格式 
#FLAGS._parse_flags()#tf 1.8版报错
FLAGS.flag_values_dict()
    


#打印参数
print("\n参数：")
#tf 1.8版报错
#for key,value in sorted(FLAGS._flags.items()):
#    print("{}={}".format(key.upper(),value))
for key in sorted(FLAGS):
    value = FLAGS[key].value
    print("{}={}".format(key.upper(),value))    
      


参数：
L2_REG_LAMBDA=0.0
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=32
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT=0.5
EMBEDDING=128
EVALUATE_EVERY=100
F=
FILTER_SIZE=3,4,5
LOG_DEVICE_PLACEMENT=True
NEGATIVE_DATA_FILE=mail/rt-polarity.neg
NUM_CHECKPOINTS=5
NUM_EPOCHS=200
NUM_FILTERS=128
POSITIVE_DATA_FILE=mail/rt-polarity.pos


In [ ]:
#==【train模块 - 读数据开始训练】==
#==读数据==
X_text, y = load_data_and_labels(FLAGS.positive_data_file,FLAGS.negative_data_file)

#预处理数据，
#邮件长短不一,不统一，特征图大小也不一样。
#所以要让所有邮件大小长度一样，让输入矩阵一样大小，
#看X_text中最大的邮件是多大，其它的padding补充
max_doc_length = max(len(X.split(' ')) for X in X_text)
vocab_processorObj = learn.preprocessing.VocabularyProcessor(max_doc_length)
X = np.array(list(vocab_processorObj.fit_transform(X_text)))#外加list
#learn.preprocessing用最大文档长度填充

#==开始训练==
np.random.seed(10)#随机种子
#洗牌 y长度（即是X_text长度，y好取）矩阵传给permutation，返回一个洗牌后的矩阵副本
shuffle_indices = np.random.permutation(np.arange(len(y)))#arange 一个r
X_shuffled = X[shuffle_indices]#index代入洗牌
y_shuffled = y[shuffle_indices]

#交叉验证 切分数据集为训练集、验证集
sample_cut_index = -1*int(FLAGS.dev_sample_percentage*float(len(y)))
X_train,X_test = X_shuffled[:sample_cut_index],X_shuffled[sample_cut_index:]#两变量可以定义在一起
y_train,y_test = y_shuffled[:sample_cut_index],y_shuffled[sample_cut_index:]
#打印
print("最大文档长度：{:d}".format(len(vocab_processorObj.vocabulary_)))
print("训练集/测试集 : {:d}/{:d}".format(len(y_train),len(y_test)))

#==tf图结构处理，执行TextCNN传数据参数==
with tf.Graph().as_default():
    #自动指派设备(cpu或gpu)
    session_conf = tf.ConfigProto(
          allow_soft_placement = FLAGS.allow_soft_placement,
          log_device_placement = FLAGS.log_device_placement
      )
      
    sess = tf.Session(config = session_conf)
      
    with sess.as_default():
        #TextCNN类实例化
        cnn = TextCNN(
              sequence_length = X_train.shape[1],
              num_classes = y_train.shape[1],
              vocab_size = len(vocab_processorObj.vocabulary_),
              embedding_size = FLAGS.embedding,
              filter_size = list(map(int,FLAGS.filter_size.split(','))),
              num_filters = FLAGS.num_filters,
              l2_reg_lambda = FLAGS.L2_reg_lambda
          )
        #X_train.shape[1]文档长度，
        #分类数y_train.shape[1] labels 有2列
        #vocab_size处理好的size是多大,文本最大长度
        #embedding_size隐层维度
        #filter_size卷积窗口尺寸,
        #FLAGES.fitler_size.split(',')上面逗号定义的3,4,5,每次卷积单词数，
        #num_filters filter数（特征图数量）
        #l2_reg_lambda正则惩罚项
      
      
        #训练模型
        #定义step
        global_step = tf.Variable(0,name='global_step')
        #实例化优化器train.Adam，梯度下降也可以，
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars,global_step)

        #==保存模型相关==
        #======
        #保存模型
        saver = tf.train.Saver(tf.global_variables(),max_to_keep = FLAGS.num_checkpoints)
        #tf.global_variables() 报 No variables to save


        #run变量初始化
        sess.run(tf.global_variables_initializer())

        #定义train_step函数
        def train_step(x_batch,y_batch):
            feed_dict = {
                cnn.input_x : x_batch,
                cnn.input_y : y_batch,
                cnn.dropout_keep_prob : FLAGS.dropout
            }

            _,step,loss,accuracy = sess.run(
                [train_op,global_step,cnn.loss,cnn.accuracy],
                feed_dict
            )
            #summaries,
            #train_summary_op,未定义，先注释
            time_str = datetime.datetime.now().isoformat()
            print("{}:step {},loss {:g}, acc {:g}".format(time_str,step,loss,accuracy))


        #定义dev_step函数 测试验证集(上面粘下来dropout改为1.0即可)
        def dev_step(x_batch,y_batch):
            feed_dict = {
                cnn.input_x : x_batch,
                cnn.input_y : y_batch,
                cnn.dropout_keep_prob : 1.0
            }

            _,step,loss,accuracy = sess.run(
                [train_op,global_step,cnn.loss,cnn.accuracy],
                feed_dict
            )
            #summaries,
            #train_summary_op,
            time_str = datetime.datetime.now().isoformat()
            print("{}:step {},loss {:g}, acc {:g}".format(time_str,step,loss,accuracy))
        
        #执行迭代函数batch_iter
        batches = batch_iter(list(zip(X_train,y_train)),FLAGS.batch_size,FLAGS.num_epochs)#list(zip(x,y))
        #【【注意：本步非常重要，要zip打包，整体组成list，变成可迭代的训练集测试集，
        #才能用next_batch（或用自己定义的其它分step迭代函数）向下迭代执行
        #自己做的数据也要用这种方式zip起来再feed给tf才能用于迭代训练,
        #本例数据路径参数已定义在tf DEFINE中，若需单独feed可考虑zip组装数据，
        #或用tf DEFINE定义数据路径
        #whale4_day6及其它分类数据衔接那里可参考本段总结】】    

        #遍历batches
        for batch in batches:
            x_batch,y_batch = zip(*batch)
            train_step(x_batch,y_batch)
            current_step = tf.train.global_step(sess,global_step)#按步sess（训练）
            #global_step 和next_batch还有计数器n = n+1 差不多，走步用的，在训练中是计数的作用，每训练一个batch就加1
            if current_step % FLAGS.evaluate_every == 0:#按之前设置的间隔条件验证测试
                print("\n evaluate_every验证测试")
                dev_step(X_test,y_test)
            if current_step % FLAGS.checkpoint_every == 0:#按条件保存结果
                path = saver.save(sess,'./textCnnModel',global_step = current_step)
                print('model模型已保存')


#注意：保存模型和下面的迭代都要放到两层width
#with tf.Graph().as_default():
#	with sess.as_default():
#		里面
#两个def函数 train_step dev_step定义在使用该函数的上面

#===看训练结果可知==
'''
#step 101,loss 0.760483, acc 0.579737
#step 2401,loss 0.258065, acc 0.904315
#step 4701,loss 0.0492993, acc 0.983114
#step 5401,loss 0.030173, acc 0.990619
#文本分类模型可用
'''




In [ ]:
'''
卷积运算的定义、动机（稀疏权重、参数共享、等变表示）。一维卷积运算和二维卷积运算。

卷积运算: 与经典的全连接神经网络相比，卷积神经网络运算的原理是通过加入卷积层和池化层，
来对特征进行逐步细化浓缩提取。
实现方式是通过滑动窗口的方式，在原始数据中划分出一个小区域矩阵，
在卷积层conv层与通过窗口filter取前一层特征图上的小区域与下一层神经元做内积和操作(即wx+b)，
经激活函数将线性转为非线性结果，
将结果，即提取的特征点汇到一个特征图，
再通过池化层pooling层 根据求max或mean操作，保留一个特征数，将特征图浓缩，
再经过下一层卷积，池化，逐步浓缩，细化特征，
最后经过两个全连接层整合特征，再softmax把特征分类。
filter在取前一层数据小窗口时，是滑动向前依次提取，
由stride指定每次滑动多远。

稀疏权重：又叫稀疏连接，就是指上面说到的，每次只关注和处理视觉范围前后的影象。
稀疏即散布离散，相对非连续性的一个概念。

参数共享：因为每个特征图小窗口在与下一层神经元做内积和操作进行wx+b操作时都带有一个w,
有多少个神经元就有多少个w，把相同的w共享避免大量操作，这个过程叫参数共享

等变表示: 等比变化，可以通过局部特性按照规律，
映射出整体或其它部分（如通过正脸特征映射画出侧脸）的特征，
反之也可以从整体抽离局部特征浓缩代表整体，等变特性是上面卷积得以实现的依据。

一维卷积运算：比如图象RGB通道，只有一个，灰度图，算上面卷积运算时只对一个平面，
做filter过滤小窗口提取特征，再pooling压缩等操作。

二维护卷积运算：chanel加入一个深度，比如图片识别的特征提取，一张彩色图，
有三个颜色通道RGB，即，输入就是 长X宽X3,3就是深度。
后面对应的filter也是三个分别提取三个通道的特征。

反卷积(tf.nn.conv2d_transpose)：
反卷积操作是卷积的反向，指上面conv pooling 一系列完成得到最终结果后，
反向传播回来求梯度使loss操失最小，来取最合适的参数w,b的操作。

tensorflow的conv2d_transpose函数可以实现反卷积，参数介绍如下：
tf.nn.conv2d_transpose(
value, #输入值，需要反卷积的输入，即从后一层反传回来的梯度值，要求tensor格式
filter, #卷积核，tensor格式，四维[f高，f宽，f个数，chanel深度或图像通道数]
output_shape, #输出的shape
strides, #滑动尺度，每一维滑动的步长
padding="SAME", #边缘填充：滑动窗口到边缘，有空出位补0操作，为了提取完整特征。VALID是舍弃这部分特征。
data_format="NHWC", #输入参数的格式，默认NHWC顺序，即[batch, height, width, in_channels]
)

池化运算的定义、种类（最大池化、平均池化等）、动机。

池化运算: 池化操作是在卷积后面跟着的一层将特征图filter提取的小区域按照规则，
取最大Max或取平均Mean来 保留一个代表性矩阵值。
动机：对特征进一步压缩细化。
这一步和卷积层的实现原理一样，只是没有wx+b 内积和 操作。
最大池化：Max，取前一层特征矩阵中最大值，如[[1,2][4,5]] 只取5放入特征图中。
平均池化：mean,前一层特征矩阵中多个值取平均产，如[[1,2][4,5]] 取 （1+2+4+5）/4 = 6

Text-CNN的原理:

TextCNN 是利用卷积神经网络对文本进行分类的算法。
卷积具有局部特征提取的功能, 所以可用 CNN 来提取句子中类似 n-gram （中心词前后的词组对）的关键信息。

Text-CNN与CNN处理图象的对比：

处理图像数据，CNN的filter卷积核宽高一致，自己指定
但Text-CNN 因为是以词为最小颗粒滑动，所以filter滑动窗口的宽度（卷积核宽），
不能自己指定（不能把一个词像图片一个拆碎了就没有意义了），
所以Text-CNN filter宽度要与词向量的维度一致，之前输入的每一行向量代表一个词，
高度可以自行指定，filter滑动卷积时提取特征时考虑了词义同时考虑了词序及其上下文,
相当于取n-gram操作（类似CBOW 由上下文算中心词概率,skip-gram 由中心词算上下文思想），
可以使用多个filter，设置不同的高度，提取多重特征 （比如使用多组filter等价于 uni-gram bi-gram tri-gram操作）。

上面是卷积层，卷积后relu激活函数转线性为非线性数据，
池化层Text-CNN是取一个Max，最重要的特征向量，

再下一层conv,relu,pooling……

最后在全连接层把每个特征值拼接起来(前面加一层dropout防过拟和)。

再经过softmax分类。

'''